### Rideshare Simulation

## Overview
- Riders and drivers are placed randomly across all available zones with equal probability.
- The matching algorithm is designed to find the nearest available driver for a rider's ride request. The driver must be in the 'idle' state and willing to accept the ride based on their preferences and the ride's metadata.

## Matching 

### 1. **Sort drivers**
First, the algorithm sorts all drivers based on their distance from the rider's current zone. This sorting is done using the Manhattan distance between the driver's zone and the rider's zone. This ensures that the closest drivers are considered first.

### 2. **Iterate through drivers**
Next, the algorithm iterates through the sorted list of drivers. For each driver, it performs the following checks:

#### a. **Check driver's state**
The driver must be in the 'idle' state, meaning they are available to take a new ride.

#### b. **Calculate & compare acceptance threshold**
The driver calculates an acceptance threshold for the ride based on their preferences and the ride's metadata. This threshold is determined by a weighted sum of the ride's attributes (cost, distance, duration, driver rating, and rider rating). The calculated acceptance threshold is compared with the driver's predefined acceptance threshold. If the calculated threshold is greater than or equal to the driver's acceptance threshold, the driver is willing to accept the ride.

### 3. **Assign driver**
If an idle driver willing to accept the ride is found, the algorithm returns this driver along with the calculated travel time to the rider's zone. If no such driver is found, the ride request may need to be re-evaluated with modified parameters (e.g., increased cost) until a suitable driver is found.

---  

The rideshare simulation consists of three main classes: `Zone`, `Driver`, `Rider`, and `RideRequest`. These classes work together to simulate the rideshare system.

In [1]:
from utils import calculate_travel_time, find_nearest_driver

from datetime import datetime, timedelta
from transitions import Machine
import networkx as nx
import random

In [2]:
# The Grid class encapsulates the creation of the directed weighted graph, zones, driver preferences, 
# and precomputes the shortest paths for a rideshare simulation. It provides an easy way to customize 
# the grid layout, edge weights, and other parameters
class Grid:
    def __init__(self, width=4, height=4, weight=2):
        self.width = width
        self.height = height
        self.weight = weight
        self.graph = nx.DiGraph()
        self.zones = {}
        self.shortest_paths = None
        self.create_graph()
        self.create_zones()
        self.precompute_shortest_paths()

    def create_graph(self):
        for i in range(self.width):
            for j in range(self.height):
                self.graph.add_node((i, j))
        
        for i in range(self.width):
            for j in range(self.height - 1):
                self.graph.add_edge((i, j), (i, j + 1), weight=self.weight)
                self.graph.add_edge((i, j + 1), (i, j), weight=self.weight)
                self.graph.add_edge((j, i), (j + 1, i), weight=self.weight)
                self.graph.add_edge((j + 1, i), (j, i), weight=self.weight)

    def create_zones(self):
        self.zones = {(i, j): Zone(i, j) for i in range(self.width) for j in range(self.height)}

    def precompute_shortest_paths(self):
        self.shortest_paths = dict(nx.all_pairs_dijkstra_path_length(self.graph, weight='weight'))
        
# Define the Zone class
class Zone:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.name = f"Zone ({x},{y})"
        
    def __str__(self):
        return f"Zone ({self.x},{self.y})"


The `Driver` class represents a driver in the rideshare system. Each driver is associated with a specific `Zone` and has preferences for accepting ride requests. The driver's state is managed by a state machine, which transitions between different states such as 'idle', 'en_route', 'waiting_for_rider', 'in_ride', and 'ride_completed'.

In [3]:
# Define the Driver class with state machine
class Driver:
    states = ['idle', 'en_route', 'waiting_for_rider', 'in_ride', 'ride_completed']

    def __init__(self, name, zone, preferences):
        self.name = name
        self.zone = zone
        self.preferences = preferences  # Driver's preferences for ride attributes
        self.time_to_next_zone = 0
        self.current_rider = None
        self.current_request = None
        self.machine = Machine(model=self, states=Driver.states, initial='idle')
        self.machine.add_transition('accept_ride', 'idle', 'en_route')
        self.machine.add_transition('arrive_at_pickup', 'en_route', 'waiting_for_rider')
        self.machine.add_transition('pick_up_rider', 'waiting_for_rider', 'in_ride')
        self.machine.add_transition('complete_ride', 'in_ride', 'ride_completed')
        self.machine.add_transition('reset', '*', 'idle')

    # Calculate acceptance threshold based on ride metadata and driver preferences
    def calculate_acceptance_threshold(self, ride_metadata):
        threshold = (
            ride_metadata['cost_usd'] * self.preferences['cost_weight'] +
            ride_metadata['distance_km'] * self.preferences['distance_weight'] +
            ride_metadata['duration_min'] * self.preferences['duration_weight'] +
            ride_metadata['driver_rating'] * self.preferences['driver_rating_weight'] +
            ride_metadata['rider_rating'] * self.preferences['rider_rating_weight']
        )
        return threshold

    # Determine if the driver accepts the ride based on the calculated threshold
    def will_accept_ride(self, ride_metadata):
        acceptance_threshold = self.calculate_acceptance_threshold(ride_metadata)
        return acceptance_threshold >= self.preferences['acceptance_threshold']

    def __str__(self):
        return f"{self.name} (State: {self.state}, Zone: {self.zone.name}, Time to next zone: {self.time_to_next_zone})"



The `Rider` class represents a rider in the rideshare system. Each rider is also associated with a specific `Zone` and has an estimated wait time for being matched with a driver. The rider's state is managed by a state machine, which transitions between states like 'waiting', 'matched', 'in_ride', 'ride_completed', and 'ride_canceled'.


In [4]:
# The Rider class represents a rider in the rideshare simulation. It includes the rider's current state, 
# zone, and estimated wait time. The rider's state machine handles transitions such as matching with a driver, 
# starting a ride, completing a ride, and canceling a ride. The rider interacts with the simulation by 
# waiting for a driver, getting matched, and going through the ride process until completion or cancellation.
class Rider:
    states = ['waiting', 'matched', 'in_ride', 'ride_completed', 'ride_canceled']

    def __init__(self, name, zone):
        self.name = name
        self.zone = zone
        self.estimated_wait_time = 0
        self.machine = Machine(model=self, states=Rider.states, initial='waiting')
        self.machine.add_transition('match_with_driver', 'waiting', 'matched')
        self.machine.add_transition('start_ride', 'matched', 'in_ride')
        self.machine.add_transition('complete_ride', 'in_ride', 'ride_completed')
        self.machine.add_transition('cancel_ride', '*', 'ride_canceled')

    def __str__(self):
        return f"{self.name} (State: {self.state}, Zone: {self.zone.name}, Estimated wait time: {self.estimated_wait_time})"


The `RideRequest` class represents a ride request made by a rider. It connects a `Rider` with a `Driver` and includes information about the origin `Zone`, destination `Zone`, and the `Zone` of the driver at the time of the request. The ride request also contains metadata such as distance, duration, cost, ratings, ride type, traffic conditions, and weather conditions. The state of the ride request is managed by a state machine, transitioning between states like 'requested', 'accepted', 'in_progress', 'completed', and 'canceled'.


In [5]:
# The RideRequest class encapsulates a ride request made by a rider. It includes the rider, origin zone, 
# destination zone, driver zone, and metadata such as distance, duration, cost, ratings, and ride type. 
# The class also has a state machine to handle the different states of the ride request, such as requested, 
# accepted, in progress, completed, and canceled.
class RideRequest:
    states = ['requested', 'accepted', 'in_progress', 'completed', 'canceled']

    def __init__(self, rider, origin_zone, destination_zone, driver_zone, shortest_paths):
        self.rider = rider
        self.origin_zone = origin_zone
        self.destination_zone = destination_zone
        self.driver_zone = driver_zone
        self.shortest_paths = shortest_paths
        self.metadata = self.generate_metadata()
        self.machine = Machine(model=self, states=RideRequest.states, initial='requested')
        self.machine.add_transition('accept', 'requested', 'accepted')
        self.machine.add_transition('start', 'accepted', 'in_progress')
        self.machine.add_transition('complete', 'in_progress', 'completed')
        self.machine.add_transition('cancel', '*', 'canceled')

    # Generate metadata for the ride request
    def generate_metadata(self):
        driver_to_pickup = calculate_travel_time(self.driver_zone, self.origin_zone, self.shortest_paths)
        pickup_to_destination = calculate_travel_time(self.origin_zone, self.destination_zone, self.shortest_paths)
        total_distance = driver_to_pickup + pickup_to_destination
        duration = total_distance  # 1 unit per zone
        cost = total_distance * 2.5  # Cost is based on the total distance with a multiplier
        
        return {
            "distance_km": total_distance,
            "duration_min": duration,
            "pickup_time": None,
            "dropoff_time": None,
            "cost_usd": round(cost, 2),
            "driver_rating": round(random.uniform(4.0, 5.0), 2),
            "rider_rating": round(random.uniform(4.0, 5.0), 2),
            "ride_type": random.choice(["economy", "premium", "pool"]),
            "traffic_conditions": random.choice(["normal", "heavy", "light"]),
            "weather_conditions": random.choice(["clear", "rainy", "stormy", "snowy"]),
        }

    def __str__(self):
        return f"RideRequest for {self.rider.name} (Origin: {self.origin_zone.name}, Destination: {self.destination_zone.name}, State: {self.state}, Metadata: {self.metadata})"



The simulation workflow involves the interaction between these classes. When a `Rider` makes a `RideRequest`, the system searches for an available `Driver` in the nearby `Zones`. The `Driver`'s acceptance of the ride request is determined by their preferences and the ride request metadata. If a `Driver` accepts the request, the states of the `Rider`, `Driver`, and `RideRequest` transition accordingly, simulating the progress of the ride from the origin `Zone` to the destination `Zone`.

The `Zone` class serves as a geographical context for the `Driver`s, `Rider`s, and `RideRequest`s, enabling the calculation of distances, travel times, and other location-based metrics.

Overall, the `Driver`, `Rider`, and `RideRequest` classes work together to simulate the rideshare system, with the `Zone` class providing the geographical context for their interactions.


In [6]:
# Simulation workflow
def simulate_rideshare(riders, drivers, ride_requests, shortest_paths=None, price_increment=5):
    time_unit = 0  # Initialize the time unit counter

    while any(rider.state != 'ride_completed' for rider in riders):  # Continue the simulation until all riders have completed their rides
        print(f"\nTime Unit: {time_unit}")  # Print the current time unit

        for rider in riders:  # Iterate over each rider
            if rider.state == 'waiting':  # Check if the rider is in the 'waiting' state
                ride_request = next((request for request in ride_requests if request.rider == rider), None)  # Find the ride request associated with the rider
                if ride_request:  # If a ride request is found
                    current_cost = ride_request.metadata["cost_usd"]  # Get the current cost of the ride request
                    driver, travel_time = find_nearest_driver(rider.zone, ride_request.metadata, drivers, shortest_paths)  # Find the nearest available driver and calculate the travel time
                    while not driver:  # If no driver is found, increment the cost and search again
                        current_cost += price_increment  # Increment the cost by the specified price increment
                        ride_request.metadata["cost_usd"] = current_cost  # Update the cost in the ride request metadata
                        driver, travel_time = find_nearest_driver(rider.zone, ride_request.metadata, drivers, shortest_paths)  # Search for a driver again with the updated cost

                    if driver:  # If a driver is found
                        print(f"{driver.name} will pick up {rider.name} from {rider.zone.name}. Travel time: {travel_time}")  # Print the driver and rider information
                        driver.zone = rider.zone  # Update the driver's zone to the rider's zone
                        driver.time_to_next_zone = travel_time  # Set the driver's time to the next zone based on the travel time
                        driver.current_rider = rider  # Assign the rider to the driver
                        driver.current_request = ride_request  # Assign the ride request to the driver
                        driver.accept_ride()  # Trigger the transition to the 'en_route' state for the driver
                        driver.current_request.metadata["pickup_time"] = datetime.now() + timedelta(minutes=time_unit)  # Set the pickup time in the ride request metadata
                        driver.current_request.accept()  # Trigger the transition to the 'accepted' state for the ride request
                        rider.match_with_driver()  # Trigger the transition to the 'matched' state for the rider
                        rider.estimated_wait_time = travel_time  # Set the estimated wait time for the rider based on the travel time

        for driver in drivers:  # Iterate over each driver
            print("Driver State:", driver)  # Print the current state of the driver
            if driver.time_to_next_zone > 0:  # If the driver has remaining time to reach the next zone
                driver.time_to_next_zone -= 1  # Decrement the time to the next zone
                print(f"{driver.name} is traveling. Time to next zone: {driver.time_to_next_zone}")  # Print the driver's traveling status
            elif driver.time_to_next_zone == 0:  # If the driver has reached the next zone
                if driver.is_en_route():  # If the driver is in the 'en_route' state
                    driver.arrive_at_pickup()  # Trigger the transition to the 'waiting_for_rider' state
                    driver.zone = driver.current_rider.zone  # Update the driver's zone to the rider's zone
                    print(f"{driver.name} has arrived at the pickup zone for {driver.current_rider.name}.")  # Print the driver's arrival status
                elif driver.is_waiting_for_rider():  # If the driver is in the 'waiting_for_rider' state
                    driver.pick_up_rider()  # Trigger the transition to the 'in_ride' state
                    driver.current_request.start()  # Trigger the transition to the 'in_progress' state for the ride request
                    driver.current_rider.start_ride()  # Trigger the transition to the 'in_ride' state for the rider
                    driver.time_to_next_zone = calculate_travel_time(driver.zone, driver.current_request.destination_zone, shortest_paths)  # Calculate the travel time to the destination zone
                    print(f"{driver.name} picked up {driver.current_rider.name} and is now in ride.")  # Print the driver's pickup status
                elif driver.is_in_ride():  # If the driver is in the 'in_ride' state
                    if driver.time_to_next_zone == 0:  # If the driver has reached the destination zone
                        driver.complete_ride()  # Trigger the transition to the 'ride_completed' state
                        driver.current_request.metadata["dropoff_time"] = datetime.now() + timedelta(minutes=time_unit)  # Set the dropoff time in the ride request metadata
                        driver.current_request.complete()  # Trigger the transition to the 'completed' state for the ride request
                        driver.current_rider.complete_ride()  # Trigger the transition to the 'ride_completed' state for the rider
                        print(f"{driver.name} completed the ride with {driver.current_rider.name}.")  # Print the driver's ride completion status
                        driver.current_rider = None  # Reset the driver's current rider
                        driver.current_request = None  # Reset the driver's current ride request
                        driver.reset()  # Trigger the transition to the 'idle' state for the driver
                        print(f"{driver.name} is now idle.")  # Print the driver's idle status
        time_unit += 1  # Increment the time unit counter


In [7]:
# Initialize the grid
grid = Grid(width=4, height=4, weight=2)

# Initialize 1 rider and 1 driver with specific zones and preferences
rider_zone = grid.zones[(2, 2)]
driver_zone = grid.zones[(1, 1)]
destination_zone = grid.zones[(3, 3)]
driver_preferences = {
            "cost_weight": 0.4,
            "distance_weight": 0.1,
            "duration_weight": 0.1,
            "driver_rating_weight": 0.2,
            "rider_rating_weight": 0.2,
            "acceptance_threshold": 20
        }

riders = [Rider('Rider 0', rider_zone)]
drivers = [Driver('Driver 0', driver_zone, driver_preferences)]
ride_request = RideRequest(riders[0], rider_zone, destination_zone, driver_zone, grid.shortest_paths)
ride_requests = [ride_request]

# Run the simulation
simulate_rideshare(riders, drivers, ride_requests, price_increment=5, shortest_paths=grid.shortest_paths)

AttributeError: 'Grid' object has no attribute 'driver_preferences'